In [ ]:
!pip install alpaca-py
!pip install alpaca-trade-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 15.8 MB/s eta 0:00:00
Reason for being yanked: This version includes overly restrictive multidict upper boundary disallowing multidict v6+. The previous patch version didn't have that and this is now causing dependency resolution problems for the users who have an "incompatible" version pinned. This is not really necessary anymore and will be addressed in the next release v3.8.3

https://github.com/aio-libs/aiohttp/pull/6950
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.5
    Uninstalling msgpack-1.0.5:

In [ ]:
import os
import alpaca_trade_api as tradeapi
import pandas as pd
import smtplib
import time
from alpaca.data import StockHistoricalDataClient, TimeFrame
from alpaca.data.requests import StockQuotesRequest, StockBarsRequest




ALPACA_API_KEY_ID = 'PKG9S8IOTW1EL9C6U5V2'
ALPACA_API_SECRET_KEY = 'zYyfTggnaMAnpkHoTbTt3whsLZq06Hf3XV7QSF2D'
api = tradeapi.REST('PKG9S8IOTW1EL9C6U5V2','zYyfTggnaMAnpkHoTbTt3whsLZq06Hf3XV7QSF2D', base_url='https://paper-api.alpaca.markets')

# Instantiate a data client
data_client = StockHistoricalDataClient(ALPACA_API_KEY_ID, ALPACA_API_SECRET_KEY)

# Set the start time
start_time = pd.to_datetime("2023-07-21").tz_localize('America/New_York')

# It's generally best to explicitly provide an end time but will default to 'now' if not
request_params = StockBarsRequest(
    symbol_or_symbols=['ADBE'],
    timeframe=TimeFrame.Minute,
    start=start_time
    )

stock1_barset = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)

# It's generally best to explicitly provide an end time but will default to 'now' if not
request_params = StockBarsRequest(
    symbol_or_symbols=['AAPL'],
    timeframe=TimeFrame.Minute,
    start=start_time
    )

stock2_barset = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)


#Selection of stocks
stock1 = 'ADBE'
stock2 = 'AAPL'
#Put Hisrorical Data into variables
stock1_bars = stock1_barset
stock2_bars = stock2_barset
stock1_bars.reset_index(inplace=True)
stock2_bars.reset_index(inplace=True)
days = len(stock1_bars)
#Grab stock1 data and put in to a array
data_1 = []
times_1 = []
for i in range(days):
    stock1_close = stock1_bars.iloc[i].close
    stock1_time = stock1_bars.iloc[i].timestamp
    data_1.append(stock1_close)
    times_1.append(stock1_time)
#Grab stock2 data and put in to an array
data_2 = []
times_2 = []
for i in range(days):
    stock2_close = stock2_bars.iloc[i].close
    stock2_time = stock2_bars.iloc[i].timestamp
    data_2.append(stock2_close)
    times_2.append(stock2_time)
#Putting them together
hist_close = pd.DataFrame(data_1, columns=[stock1])
hist_close[stock2] = data_2
#Current Spread between the two stocks
stock1_curr = data_1[days-1]
stock2_curr = data_2[days-1]
spread_curr = (stock1_curr-stock2_curr)
#Moving Average of the two stocks
move_avg_days = 5
#Moving averge for stock1
stock1_last = []
for i in range(move_avg_days):
    stock1_last.append(data_1[(days-1)-i])

stock1_hist = pd.DataFrame(stock1_last)

stock1_mavg = stock1_hist.mean()
#Moving average for stock2
stock2_last = []
for i in range(move_avg_days):
    stock2_last.append(data_2[(days-1)-i])
stock2_hist = pd.DataFrame(stock2_last)
stock2_mavg = stock2_hist.mean()
#Sread_avg
spread_avg = min(stock1_mavg - stock2_mavg)
#Spread_factor
spreadFactor = .01
wideSpread = spread_avg*(1+spreadFactor)
thinSpread = spread_avg*(1-spreadFactor)
#Calc_of_shares_to_trade
account = api.get_account()
cash = float(account.buying_power)
limit_stock1 = cash//stock1_curr
limit_stock2 = cash//stock2_curr
number_of_shares = int(min(limit_stock1, limit_stock2)/2)

#Trading_algo
portfolio = api.list_positions()
clock = api.get_clock()

while clock.is_open == True:

    ALPACA_API_KEY_ID = 'PKG9S8IOTW1EL9C6U5V2'
    ALPACA_API_SECRET_KEY = 'zYyfTggnaMAnpkHoTbTt3whsLZq06Hf3XV7QSF2D'
    api = tradeapi.REST('PKG9S8IOTW1EL9C6U5V2','zYyfTggnaMAnpkHoTbTt3whsLZq06Hf3XV7QSF2D', base_url='https://paper-api.alpaca.markets')

    portfolio = api.list_positions()

    # Instantiate a data client
    data_client = StockHistoricalDataClient(ALPACA_API_KEY_ID, ALPACA_API_SECRET_KEY)

    # Set the start time
    start_time = pd.to_datetime("2023-07-21").tz_localize('America/New_York')

    # It's generally best to explicitly provide an end time but will default to 'now' if not
    request_params = StockBarsRequest(
        symbol_or_symbols=['ADBE'],
        timeframe=TimeFrame.Minute,
        start=start_time
        )

    stock1_barset = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)

    # It's generally best to explicitly provide an end time but will default to 'now' if not
    request_params = StockBarsRequest(
        symbol_or_symbols=['AAPL'],
        timeframe=TimeFrame.Minute,
        start=start_time
        )

    stock2_barset = data_client.get_stock_bars(request_params).df.tz_convert('America/New_York', level=1)


    #Selection of stocks
    stock1 = 'ADBE'
    stock2 = 'AAPL'
    #Put Hisrorical Data into variables
    stock1_bars = stock1_barset
    stock2_bars = stock2_barset
    stock1_bars.reset_index(inplace=True)
    stock2_bars.reset_index(inplace=True)
    days = len(stock1_bars)
    #Grab stock1 data and put in to a array
    data_1 = []
    times_1 = []
    for i in range(days):
        stock1_close = stock1_bars.iloc[i].close
        stock1_time = stock1_bars.iloc[i].timestamp
        data_1.append(stock1_close)
        times_1.append(stock1_time)
    #Grab stock2 data and put in to an array
    data_2 = []
    times_2 = []
    for i in range(days):
        stock2_close = stock2_bars.iloc[i].close
        stock2_time = stock2_bars.iloc[i].timestamp
        data_2.append(stock2_close)
        times_2.append(stock2_time)
    #Putting them together
    hist_close = pd.DataFrame(data_1, columns=[stock1])
    hist_close[stock2] = data_2
    #Current Spread between the two stocks
    stock1_curr = data_1[days-1]
    stock2_curr = data_2[days-1]
    spread_curr = (stock1_curr-stock2_curr)
    #Moving Average of the two stocks
    move_avg_days = 5
    #Moving averge for stock1
    stock1_last = []
    for i in range(move_avg_days):
        stock1_last.append(data_1[(days-1)-i])

    stock1_hist = pd.DataFrame(stock1_last)

    stock1_mavg = stock1_hist.mean()
    #Moving average for stock2
    stock2_last = []
    for i in range(move_avg_days):
        stock2_last.append(data_2[(days-1)-i])
    stock2_hist = pd.DataFrame(stock2_last)
    stock2_mavg = stock2_hist.mean()
    #Sread_avg
    spread_avg = min(stock1_mavg - stock2_mavg)
    #Spread_factor
    spreadFactor = .01
    wideSpread = spread_avg*(1+spreadFactor)
    thinSpread = spread_avg*(1-spreadFactor)
    #Calc_of_shares_to_trade
    account = api.get_account()
    cash = float(account.buying_power)
    limit_stock1 = cash//stock1_curr
    limit_stock2 = cash//stock2_curr
    number_of_shares = int(min(limit_stock1, limit_stock2)/2)

    if bool(portfolio) == False:
        #detect a wide spread

        if spread_curr > wideSpread:
            #short top stock
            api.submit_order(symbol = stock1,qty = number_of_shares,side = 'sell',type = 'market',time_in_force ='day')
            #Long bottom stock
            api.submit_order(symbol = stock2,qty = number_of_shares,side = 'buy',type = 'market',time_in_force = 'day')

            print("Trades have been made, short top stock and long bottom stock")
        #detect a tight spread
        elif spread_curr < thinSpread:
            #long top stock

            api.submit_order(symbol = stock1,qty = number_of_shares,side = 'buy',type = 'market',time_in_force = 'day')
            #short bottom stock
            api.submit_order(symbol = stock2,qty = number_of_shares,side = 'sell',type = 'market',time_in_force ='day')

            print("Trades have been made, long top stock and short bottom stock")

    else:
        wideTradeSpread = spread_avg *(1+spreadFactor + .03)
        thinTradeSpread = spread_avg *(1-spreadFactor - .03)
        if spread_curr <= wideTradeSpread and spread_curr >=thinTradeSpread:
            api.close_position(stock2)
            api.close_position(stock1)
            print("Position has been closed")

        else:
            print("No trades were made, position remains open")
    time.sleep(3)
else:
    print("The Market is Closed")








Position has been closed


KeyboardInterrupt: ignored